In [1]:
import pandas as pd
import math
from time import sleep
from opencage.geocoder import OpenCageGeocode
from clean import *

from emoji import UNICODE_EMOJI

from nmf_helpers import build_text_vectorizer, hand_label_topics, analyze_article
from my_nmf import NMF

from scipy import spatial

with open('data/loc_data.json','r') as file:
    location_dict = json.load(file)


In [2]:
data = pd.read_csv('data/raw/profile_data.csv')

In [3]:
data.head()

,name,age,college,job,city,gender,distance,details,anthem,profile_pic_urls
0,gianna,19.0,NaN,NaN,Lives in Barnegat,NaN,2430 miles away,ig- _.giaannaa_\r\nsc- gianna8378,"('Slide (Remix) (feat. Pop Smoke, A Boogie Wit...",['https://images-ssl.gotinder.com/5dc0814dfc09...
1,lauren,21.0,Florida State University,NaN,NaN,NaN,1981 miles away,A meme,NaN,['https://images-ssl.gotinder.com/5e8a352520cc...
2,Britanny,19.0,Santa Ana College,Boys and girls club,Lives in Garden Grove,NaN,19 miles away,Get to know me ☺️ Never been to Disneyland 😅 S...,NaN,['https://images-ssl.gotinder.com/5cace3ad9b7a...
3,Kayla,28.0,School,NaN,Lives in Los Angeles,Woman,18 miles away,"I am a creative person that loves simple, inno...",NaN,['https://images-ssl.gotinder.com/5e90cb67da1c...
4,Kimberly,18.0,"California State University, Los Angeles",NaN,NaN,Woman,15 miles away,Just for fun\r\nI’m 5’2,NaN,['https://images-ssl.gotinder.com/5e0ffa53349f...


In [4]:
data = clean(data)

-----------------
CLEANING DATA
...
FOUND 118396 ENTRIES
REMOVED 40960 EMPTY ENTRIES
REMOVED 1224 DUPLICATED ENTRIES


In [5]:
data.head()

,name,age,college,job,city,gender,distance,details,anthem,profile_pic_urls,num_pics,filtered_details
0,Gianna,19.0,NaN,NaN,Barnegat,NaN,2430.0,ig- _.giaannaa_\r\nsc- gianna8378,"('Slide (Remix) (feat. Pop Smoke, A Boogie Wit...",['https://images-ssl.gotinder.com/5dc0814dfc09...,4,"{sc-, ig-, _.giaannaa_, gianna8378}"
1,Lauren,21.0,Florida State University,NaN,NaN,NaN,1981.0,A meme,NaN,['https://images-ssl.gotinder.com/5e8a352520cc...,4,{meme}
2,Britanny,19.0,Santa Ana College,Boys and girls club,Garden Grove,NaN,19.0,Get to know me ☺️ Never been to Disneyland 😅 S...,NaN,['https://images-ssl.gotinder.com/5cace3ad9b7a...,7,"{know, get, brix207, disneyland, 😅, insta, geo..."
3,Kayla,28.0,School,NaN,Los Angeles,Woman,18.0,"I am a creative person that loves simple, inno...",NaN,['https://images-ssl.gotinder.com/5e90cb67da1c...,1,"{genres, looks, blend, loves, like, music, inn..."
4,Kimberly,18.0,"California State University, Los Angeles",NaN,NaN,Woman,15.0,Just for fun\r\nI’m 5’2,NaN,['https://images-ssl.gotinder.com/5e0ffa53349f...,6,"{’, 5, 2, fun}"


In [6]:
stats(data)

-----------------
CALCULATING STATS
...
FOUND 76212 ENTRIES
75609 ENTRIES HAVE AGE DATA (99.2%)
26468 ENTRIES HAVE COLLEGE DATA (34.7%)
18958 ENTRIES HAVE CITY DATA (24.9%)
18822 ENTRIES HAVE JOB DATA (24.7%)
13920 ENTRIES HAVE GENDER DATA (18.3%)
74614 ENTRIES HAVE DISTANCE DATA (97.9%)
56515 ENTRIES HAVE DETAIL DATA (74.2%)
19925 ENTRIES HAVE ANTHEM DATA (26.1%)
...
AVERAGE AGE IN DATA SET: 24
NUMBER OF UNIQUE COLLEGES: 9719
NUMBER OF UNIQUE JOBS: 11960
NUMBER OF UNIQUE CITIES: 3413
AVERAGE PICS PER PROFILE: 5


In [7]:
data = add_topics(data)

In [8]:
data.head()

,name,age,college,job,city,gender,distance,details,anthem,profile_pic_urls,...,filtered_details,instagram,snapchat,premium snap,covid19,cannabis,dirty,tv,office,himym
0,Gianna,19.0,NaN,NaN,Barnegat,NaN,2430.0,ig- _.giaannaa_\r\nsc- gianna8378,"('Slide (Remix) (feat. Pop Smoke, A Boogie Wit...",['https://images-ssl.gotinder.com/5dc0814dfc09...,...,"{sc-, ig-, _.giaannaa_, gianna8378}",True,True,False,False,False,False,False,False,False
1,Lauren,21.0,Florida State University,NaN,NaN,NaN,1981.0,A meme,NaN,['https://images-ssl.gotinder.com/5e8a352520cc...,...,{meme},False,False,False,False,False,False,False,False,False
2,Britanny,19.0,Santa Ana College,Boys and girls club,Garden Grove,NaN,19.0,Get to know me ☺️ Never been to Disneyland 😅 S...,NaN,['https://images-ssl.gotinder.com/5cace3ad9b7a...,...,"{know, get, brix207, disneyland, 😅, insta, geo...",True,True,False,False,False,False,False,False,False
3,Kayla,28.0,School,NaN,Los Angeles,Woman,18.0,"I am a creative person that loves simple, inno...",NaN,['https://images-ssl.gotinder.com/5e90cb67da1c...,...,"{genres, looks, blend, loves, like, music, inn...",False,False,False,False,True,False,False,False,False
4,Kimberly,18.0,"California State University, Los Angeles",NaN,NaN,Woman,15.0,Just for fun\r\nI’m 5’2,NaN,['https://images-ssl.gotinder.com/5e0ffa53349f...,...,"{’, 5, 2, fun}",False,False,False,False,False,False,False,False,False


In [32]:
data['office'].sum()

252

In [33]:
for each in data[data['office']==True].details.to_list():
    print(each)


Please be funny 🙃 Looking for someone that's tolerant of me watching The Office over and over again. 
MS in Counseling and ABA
Eternally sarcastic 
It’s a quarterly tradition of mine to rewatch the Office 
Here to date, I have enough pen pals 🤷🏾‍♀️ You’ll know I like you if I offer to cook for you
I adore Halloween... anything spooky really 
Hello world, 
Tinder open the borders virtual, so i travel online. Im from germany. Lets exchange music or filmtipps or have nice talks. Take care. 

En/Ger

Music
Jayda G, Kiddy Smile, the blaze, Queen, Prince, David Bowie, Björk, Lila Downs, yesayer, Anthony Hamilton, xx, Beach House, Telemann, Tal R, Vök, Rhye, Tank and the Bangers, Billy Idol, Sade, Jill Scott, Common, Stephanie Mills, Fleetwood Mac, Michael Kiwanuka,
Sundan Archives, Cigarettes After Sex
... 🏳️‍🌈 Ukkodemakka insta 
Love the outdoors 🌻 🐎 Even though imma city girl 🌃 🌇 I ❤ animals more than people 🦁 🦓 🦜 🦄 😘 Office Expert 📝 I like a man with a plan. Hmu with a date idea already i

In [11]:
len(data[data.snapchat==False].filtered_details.to_list())

51616

In [14]:
len(data.filtered_details.to_list())

76212

In [29]:
test = data.iloc[2,:].details

In [33]:
bad_chars = ["\n","\r"]

In [36]:
for each in bad_chars:
    test = test.replace(each,"")

In [37]:
test

'Get to know me ☺️ Never been to Disneyland 😅 SC:brix207 Insta: @georgely_ '

In [43]:
def details_stats(details):
    data = {}
    details = details.lower()
    words = details.split(' ')
    data['words'] = len(words)
    
    return data

In [44]:
a = details_stats(test)

In [45]:
a

{'words': 14}

In [51]:
'☺️' in test

True

In [55]:
instagram_tags = ['ig','insta','instagram','📷','📸']
snapchat_tags = ['sc','snap','snapchat','👻']
premium = ['venmo','cashapp','💰']
coronavirus = ['corona','coronavirus','quarantine','covid','covid-19','wfh','work from home','social distancing']

In [152]:
topics = {}
topics['instagram'] = ['ig','insta','instagram','📷','📸','@']
topics['snapchat'] = ['sc','snap','snapchat','👻']
topics['premium snap'] = ['venmo','cashapp','💰']
topics['covid19'] = ['corona','coronavirus','cv','quarantine','covid','covid19','virus','sick','covid-19','wfh','work from home','social distancing','shelter','apocalypse','2020']
topics['cannabis'] = ['weed','420','maryjane','smoke','blaze','marijuana','blunt','bong','joint','high','drug','drugs','cannabis']
topics['dirty'] = ['pussy','dick','penis','vagina','daddy','🍆','👅','💦','fuck','sex']
topics['tv'] = ['netflix','tv','show','hulu','movies','movie','watch','series']
topics['office'] = ['office','jim','pam','dwight','schrute','michael','dunder','mifflin']
topics['himym'] = ['himym']


In [95]:
def word_check(details):
    if type(details)==float:
        return []
    else:
        details = details.lower()
        mentions = []
        for each in topic:
            for word in topic[each]:
                if word in details:
                    mentions.append(each)
                    break
        return mentions
            

In [ ]:
def topic_check(details,topic):
    for word in topics[topic]:
        if word in set(details):
            return True
    return False
            

In [ ]:
def topic_check(filtered_details,topic):
    for word in topics[topic]:
        if word in filtered_details:
            return True
    return False

def add_topics(data):
    for each in topics:
        data[each] = data[-data.filtered_details.isna()].apply(lambda x: topic_check(x,each))
    return data

In [96]:
word_check(test)

['instagram', 'snapchat']

In [97]:
data['mentions'] = data.details.apply(lambda x: word_check(x))

In [98]:
data['instagram'] = data.mentions.apply(lambda x: True if 'instagram' in x else False)

In [102]:
data['snapchat'] = data.mentions.apply(lambda x: True if 'snapchat' in x else False)

In [101]:
data.instagram.sum()

14923

In [103]:
data.snapchat.sum()

6531

In [111]:
6531/56783

0.11501681841396193

In [110]:
for each in data[[data.snapchat==False]and[data.instagram==False]].details.tolist():
    print(each)

ValueError: Item wrong length 1 instead of 56783.

In [137]:
contents = data[-data.details.isna()]

In [138]:
contents = contents[contents.instagram==False]

In [139]:
contents = contents[contents.snapchat==False]

In [141]:
contents = contents.details

In [179]:
words = set()
for each in contents:
    inside = each.split(' ')
    for i in inside:
        words.add(i)

In [181]:
len(words)

67650

In [143]:
vectorizer, vocabulary = build_text_vectorizer(contents,
                             use_tfidf=True,
                             use_stemmer=False,
                             max_features=5000)
X = vectorizer(contents)

In [156]:
vectorizer

<function nmf_helpers.build_text_vectorizer.<locals>.vectorizer(X)>

In [148]:
for each in vocabulary:
    print(each)

'
'19
0
000
1
10
100
11
12
13
14
15
16
160
165
165cm
167
168
169cm
17
170
170cm
172
173
173cm
174
175
177
178
18
180
19
1999
1st
2
20
2000
2000s
2015
2017
2018
2019
2020
21
22
23
24
25
26
27
28
29
2nd
3
30
31
32
33
35
37
38
3rd
4
4'11
40
420
44
4th
5
5'
5'1
5'10
5'11
5'2
5'3
5'4
5'5
5'6
5'7
5'8
5'9
50
500
54
55
5ft
5th
6
6'
6'0
60
63
64
65
67
68
6ft
7
70
72
73
73cm
74
75
76
78
8
80
80's
80s
8th
9
90
90's
90s
97
98
99
_
aan
abba
aber
ability
able
abogada
abroad
abs
absolute
absolutely
abt
aburrida
aburro
ac
acab
acc
accent
accept
accepting
access
accident
accidentally
according
accordingly
account
accountant
accounting
achieve
acho
across
act
acting
action
actions
active
activist
activities
activity
actor
actress
actriz
actual
actually
acá
add
added
addict
addicted
addiction
administración
administration
admit
adorable
adore
adoro
adrenaline
adult
adults
advance
advantage
adventure
adventurer
adventures
adventuring
adventurous
advertising
advocate
af
affection
affectionate
aficionado
af

engage
engaging
engineer
engineering
england
english
enjoy
enjoying
enjoys
enneagram
enough
enter
entertain
entertained
entertainer
entertaining
entertainment
enthusiast
enthusiastic
entiendo
entire
entj
entp
entre
entrepreneur
então
environment
environmental
episode
equal
equally
equestrian
er
era
eres
es
esfj
eso
esp
español
especial
especially
espero
esportes
espresso
espírito
esquerda
essa
esse
essential
est
esta
estamos
estar
estate
este
esteem
esto
estou
estoy
estudante
estudiando
estudiante
estudio
está
estás
et
etc
eternal
ethically
ethnicity
ett
etter
että
eu
europe
european
even
event
events
eventually
ever
every
everybody
everyday
everyone
everything
everywhere
evil
ex
exactly
excel
excellent
except
exchange
excited
excitement
exciting
excuse
executive
exercise
exercising
exist
existence
existential
exists
exotic
expand
expect
expectations
experience
experiences
experiencing
expert
explain
explore
explorer
exploring
extra
extreme
extremely
extrovert
extroverted
extrovertida


range
rap
rapper
rare
rarely
rat
rate
rather
rating
rato
rave
ravenclaw
raver
raves
ray
rbf
reach
read
reader
reading
reads
ready
real
realist
reality
realize
realized
really
reason
rec
recent
recently
recipes
recommend
recommendations
record
records
recovering
recs
red
reddit
redhead
reference
references
reggae
registered
regret
regrets
regular
regularly
reise
relacionamento
relaciones
related
relation
relations
relationship
relationships
relax
relaxation
relaxing
remember
remind
remotely
renaissance
repeat
reply
republic
requests
required
requirement
requirements
research
researcher
resident
respect
respectable
respectful
respeito
respond
responder
response
responsible
rest
restaurant
restaurants
resting
resto
retired
return
rewatching
rey
reír
ri
rica
rican
rice
rich
rick
rico
rid
ride
ridiculous
riding
rihanna
ring
rings
rio
rip
risada
rising
risk
river
rj
rly
rn
rnb
road
roading
roadtrips
roast
rob
rock
rockabilly
rocks
rocky
role
roll
roller
rollerblading
rollercoasters
rolls
rom

In [146]:
# We'd like to see consistent results, so set the seed.
np.random.seed(12345)

# Find latent topics using our NMF model.
factorizer = NMF(k=3, max_iters=5, alpha=0.5)
W, H = factorizer.fit(X, verbose=True)

iter 0 : reconstruction error: 1027.722776783484
iter 1 : reconstruction error: 293.6433050073501
iter 2 : reconstruction error: 158.95489272828357
iter 3 : reconstruction error: 146.14762289434898
iter 4 : reconstruction error: 144.84333081877742
FINAL reconstruction error: 144.17823831488775 



In [147]:
# Label topics and analyze a few NYT articles.
hand_labels = hand_label_topics(H, vocabulary)

topic 0
--> 5 looking i'm la hi someone 1 want let serious get find trying 7 2 meet send take know 8
please label this topic: looking

topic 1
--> love like fun good music make hey know let friends new laugh go brazilian life traveling always adventures travel english
please label this topic: travel

topic 2
--> quarantine bored inst time talk friends made new people need lover want tinder name girl trade pic written world like
please label this topic: quarantine



In [130]:
hand_labels

['bored',
 'social media',
 'quarantine',
 'tinder',
 'lonely',
 'friends',
 'tiktok']

In [168]:


dataSetI = X[10]
dataSetII = X[11]
result = spatial.distance.cosine(dataSetI, dataSetII)

In [169]:
result

1.0

In [167]:
X[10].sum()

4.1508915510484385

In [170]:
X[11].sum()

3.632529560340464

In [182]:
from sklearn.metrics.pairwise import cosine_similarity

In [186]:
a = cosine_similarity([X[10]],[X[11]])

In [188]:
import numpy as np

def cosine_distance(a, b):
    """Compute the cosine dissimilarity between two numpy arrays.
    Parameters
    ----------
    a: numpy array
    b: numpy array
    Returns
    -------
    distance: float
    """
    return 1 - np.dot(a, b) / np.sqrt(np.dot(a, a) * np.dot(b, b))

In [191]:
num = cosine_distance(X[10],X[110])

In [192]:
num

1.0

In [ ]:
    cosine_diff = []
    for song in vectors:
        average = []
        for song2 in vectors:
            average.append(cosine_distance(song,song2))
        cosine_diff.append(np.mean(average))

In [16]:
for each in my_set:
    print(each)

c
a
b


In [15]:
my_set = {'a','b','c'}

In [13]:
takeout = {'b','c','d'}
test = {'g'}

In [25]:
if my_set.intersection(takeout)==set():
    print(True)
else:
    print(False)

False


In [22]:
my_set.intersection(test)

set()

In [18]:
my_list = 'ig-'

In [19]:
if 'ig' in my_list:
    print("okay")

okay


In [12]:
a = list({'gianna8378', 'ig-', '_.giaannaa_', 'sc-'})

In [29]:
print(a)
thing = 'ig'
for each in a:
    print(each)
    if thing in each:
        print("okay")

['gianna8378', 'ig-', '_.giaannaa_', 'sc-']
gianna8378
ig-
okay
_.giaannaa_
sc-


In [25]:
fullstring = "StackAbuse"
substring = "tack"

if substring in fullstring:
    print("Found!")
else:
    print("Not found!")

Found!
